# Generación de texto con LSTMs

In [1]:
# Paso 1: importar librerías
import re
import numpy as np
import pandas as pd
from sklearn import preprocessing
from keras import models, layers

Using TensorFlow backend.


In [2]:
#Paso 2: importar cuento  + pasar todo a minusculas + pasar caracteres a integers
# load ascii text and covert to lowercase
f = open('MobyDick.txt', 'r', encoding="utf8") 
text = f.read()
f.close()
text = text.lower()
text = re.sub(r"[^a-zA-Z0-9]+", ' ', text)
#text = text[:100000]

# create mapping of unique chars to integers, and a reverse mapping
chars = np.array(list(text))
encoder = preprocessing.LabelEncoder().fit(chars)
encoded_chars = encoder.transform(chars)

# normalize data
mean = encoded_chars.mean()
stdv = encoded_chars.std()
encoded_chars = (encoded_chars - mean) / stdv

len(chars)

1194254

In [3]:
#paso 3: preparar datos: 
# 1- preprocesar: ejemplo: CHAPT -> E; HAPTE -> R (pero con 100 caracteres)
# split a univariate sequence into samples
def split_sequence(sequence, n_steps, n_step_diff=1):
    X, y = list(), list()
    i = 0
    while i <len(sequence): 
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        i = i + (n_step_diff)
    return np.array(X), np.array(y)

X_train, y_train = split_sequence(encoded_chars, 100, 100)

# 2- cambiar dataset para que la entrada sea válida: [Samples, time_steps,features]
n_samples = X_train.shape[0]
n_steps = X_train.shape[1] # 100
n_features = 1
X_train = X_train.reshape((n_samples, n_steps, n_features))

# 3-Normalizar
    # normalizamos todo después de usar label encoder

# prepare the dataset of input to output pairs encoded as integers

In [4]:
# Preparar datos de salida (one-hot encoder)
# one hot encode the output variable

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.40, random_state=42)

In [5]:
# Definimos modelo LSTM
lstm_model = models.Sequential([
    layers.LSTM(32, input_shape=X_train.shape[-2:]),
    layers.Dense(1)
])

lstm_model.compile(optimizer='rmsprop', loss='mse')



In [6]:
#Hacemos el FIT
history = lstm_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Train on 7165 samples, validate on 4777 samples
Epoch 1/5
7165/7165 [==============================] - 14s 2ms/step - loss: 0.9305 - val_loss: 0.9008
Epoch 2/5
7165/7165 [==============================] - 14s 2ms/step - loss: 0.9125 - val_loss: 0.8998
Epoch 3/5
7165/7165 [==============================] - 14s 2ms/step - loss: 0.9080 - val_loss: 0.8868
Epoch 4/5
7165/7165 [==============================] - 14s 2ms/step - loss: 0.9035 - val_loss: 0.8834
Epoch 5/5
7165/7165 [==============================] - 14s 2ms/step - loss: 0.8997 - val_loss: 0.8807


In [7]:
#Generar datos caracteres
# pick a random seed
